# MDIBL Transcriptome Assembly Learning Module
# Bonus notebook: Using TransPi on a new dataset

In this notebook, we are going to explore how to run this module with a new dataset. These submodules provide a great framework for running a rigorous and scalable transcriptome assembly, but there are some considerations that must be made in order to run this with your own data. We will walk through that process here so that hopefully, you are able to take these notebooks to your research group and use them for your own analysis.

The data we are using here comes from SRA. In this example, we are using data from an experiment that compared RNA sequences in honeybees with and without viral infections. The BioProject ID is [PRJNA274674](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA274674). This experiment includes 6 RNA-seq samples and 2 methylation-seq samples. We are only considering the RNA-seq data here. Additionally, we have subsampled them to about 2 millions reads collectively accross all of the samples. In a real analysis this would not be a good idea, but to keep costs and runtimes low we will use the down-sampled files in this demonstration. If you want to explore the full dataset, we recommend pulling the fastq files using the [STRIDES tutorial on SRA downloads](https://github.com/STRIDES/NIHCloudLabGCP/blob/main/tutorials/notebooks/SRADownload/SRA-Download.ipynb). As with the original example in this module, we have concatenated all 6 files into one set of combined fastq files called apis_joined_R{1,2}.fastq.gz We have stored the subsampled fastq files in this module's cloud storage bucket.

Before we start any analysis, let's set up the environment just like we did in Submodule_01 and Submodule_02 where we move to the correct directory and install software.

In [ ]:
%cd /home/jupyter

In [ ]:
!pwd

In [ ]:
#update java
!sudo apt update
!sudo apt-get install default-jdk -y
!java -version

In [ ]:
# install mamba and dependencies
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge
!~/mambaforge/bin/mamba install -c bioconda sra-tools perl-dbd-sqlite perl-dbi -y

In [ ]:
#install Nextflow
!curl https://get.nextflow.io | bash
!chmod +x nextflow
!./nextflow self-update

In [ ]:
# Copy the software from the storage bucket
!gsutil -m cp -r gs://nigms-sandbox/nosi-inbremaine-storage/TransPi ./

In [ ]:
# Copy the data from the storage bucket
!gsutil -m cp -r gs://nigms-sandbox/nosi-inbremaine-storage/resources ./

In [ ]:
#Make the program executable
!chmod -R +x ./TransPi/bin

Let's have a look at what we've downloaded to make sure it's there.

In [ ]:
!ls ./resources/seq2

You should see the apis_joined fastq files alongside the others that we use in the previous submodules. Now let's adjust the workflow to run on them.

One of the great benefits of using a workflow manager like Nextflow is that it allows easy swapping of input samples without drastic changes to the code. In the true spirit of reproducible workflows, the only change necessary in order to run the apis samples is to adjust the `reads` line in the `nextflow.config` file `params` section to point to the new reads location. In the line below, write the updated reads path that you would add to the config file. 

In [1]:
# <Your path here>

<details>
  <summary>Click for help</summary>
    


```
// Directory for reads
reads="/home/jupyter/resources/seq2/apis_joined*R[1,2].fastq.gz"
```
    
    
</details>


After this change, you should be able to run the same Nextflow command as you did in Submodule_02 and everything will progress automatically.

In [ ]:
!NXF_VER=22.10.1 ./nextflow run ./TransPi/TransPi.nf \
-profile docker --k 17,25,43 --maxReadLen 50 --all 

With the subsampled reads, the assembly should complete in about 2 hours using a n1-highmem-16 machine.